In [1]:
import pandas as pd
import warnings
import tkinter as tk
warnings.filterwarnings('ignore')


In [26]:
column_names = ['user_id','item_id','rating','timestamp']
df = pd.read_csv('u.data',sep='\t',names=column_names)
df1 = pd.read_csv('movies.csv')
df1 = df1.rename({'movieId':'item_id'},axis=1)
df = pd.merge(df,df1,on='item_id')
ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
ratings['num of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
moviemat = df.pivot_table(index='user_id',columns='title',values='rating')

app = tk.Tk()
app.title('Recommendation system')
app.geometry('350x200')
tk.Label(app,text='Movie you watched').place(x=25,y=25)
tk.Label(app,text='Recommended:').place(x=25,y=100)
tk.Label(app,text='Also try:').place(x=25,y=125)

movie_var = tk.Variable(app)
tk.Entry(app,textvariable=movie_var,width=28).place(x=150,y=25)

recommended_var = tk.Variable(app)
recommended_var.set('Type movie name')
tk.Label(app,textvariable=recommended_var).place(x=150,y=100)

also_try_var = tk.Variable(app)
also_try_var.set('Type movie name..')
tk.Label(app,textvariable=also_try_var).place(x=150,y=125)

def find_recommendation():
    movie = movie_var.get().lower().strip()
    try:
        movie = df1['title'][df1['title'].apply(lambda x: movie in x.lower())].values[0]
        movie_var.set(movie)
    except IndexError:
        recommended_var.set('Movie not found')
        also_try_var.set('Movie not found')
    else:
        corr = pd.DataFrame(moviemat.corrwith(moviemat[movie]),columns=['Correlation'])
        corr.dropna(inplace=True)
        corr = corr.join(ratings['num of ratings'])
        recommended_var.set(corr[(corr['num of ratings']>100)&(corr['Correlation']>0.4)].sort_values('Correlation',ascending=False).index[1])
        also_try_var.set(corr[corr['Correlation']>0.4].sort_values('Correlation',ascending=False).index[0])
        

tk.Button(app,text='Find Suggestion',command=find_recommendation).place(x=125,y=60)

app.mainloop()